# Merge of API and Booking.com

In [ ]:
#pip install fuzzywuzzy
#pip install python-Levenshtein

In [ ]:
import csv
import pandas as pd
import math

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
api_structure = pd.read_csv('api_structure.csv', encoding='utf-8', header=0)
booking_structure = pd.read_csv('booking_base.csv', encoding='utf-8', header=0)

In [ ]:
# Print the first rows of the file "api_structure.csv"
print("Prime righe di api_structure.csv:")
api_structure.head()

In [ ]:
# Print the number of rows of the file "api_structure.csv"
print("Numero di righe di api_structure.csv:", api_structure.shape[0])
print("Numero di colonne di api_structure.csv:", api_structure.shape[1])

In [ ]:
# Print the first rows of the file "booking_base.csv"
print("Prime righe di booking_base.csv:")
booking_structure.head()

In [ ]:
# Print the number of rows of the file "booking_base.csv"
print("Numero di righe di booking_base.csv:", booking_structure.shape[0])
print("Numero di colonne di booking_base.csv:", booking_structure.shape[1])

### Work on each dataset

In [ ]:
# Count the number of rows for each 'comune' in 'api_structure'
api_structure_counts = api_structure['nome_comune'].value_counts()

# Count the number of rows for each 'comune' in 'booking_structure'
booking_structure_counts = booking_structure['comune'].value_counts()

In [ ]:
api_structure_counts

In [ ]:
# Substitute all the occurencies of "TEMÃ™" with "Temù" in "nome_comune"
api_structure['nome_comune'] = api_structure['nome_comune'].replace('TEMÃ™', 'Temù')

# count the number of rows for each 'comune' in 'api_structure'
api_structure_counts = api_structure['nome_comune'].value_counts()

api_structure_counts

In [ ]:
booking_structure_counts

In [ ]:
# Rename columns to avoid errors
api_structure = api_structure.rename(columns={'nome_comune': 'API_Comune', 'indirizzo': 'API_Indirizzo'})
booking_structure = booking_structure.rename(columns={'comune': 'Booking_Comune', 'indirizzo': 'Booking_Indirizzo'})

# Transforms both columns to lower case to perform comparison without case difference
api_structure['API_Comune'] = api_structure['API_Comune'].str.lower()
api_structure['API_Indirizzo'] = api_structure['API_Indirizzo'].str.lower()
booking_structure['Booking_Comune'] = booking_structure['Booking_Comune'].str.lower()
booking_structure['Booking_Indirizzo'] = booking_structure['Booking_Indirizzo'].str.lower()

In [ ]:
# Merge the datasets on the 'comune' column.
merged_data = pd.merge(api_structure, booking_structure, left_on="API_Comune", right_on="Booking_Comune", how="inner")

# Combine the values of columns"API_Comune" and "Booking_Comune" into a single column "Comune"
# If API_Comune is missing then the dataset will use Booking_Comune to make up for this lack.
merged_data["Comune"] = merged_data["API_Comune"].combine_first(merged_data["Booking_Comune"])

# Remove the original "API_Comune" and "Booking_Comune"
merged_data = merged_data.drop(["API_Comune", "Booking_Comune"], axis=1)

# Set the option to display all columns
pd.set_option('display.max_columns', None)
merged_data

In [ ]:
# create 3 distinct dataframes for 3 distinct cases that I can get

equal_address = merged_data[(merged_data['API_Indirizzo'] == merged_data['Booking_Indirizzo'])]
different_address = merged_data[(merged_data['API_Indirizzo'].notnull()) & (merged_data['Booking_Indirizzo'].notnull()) & (merged_data['API_Indirizzo'] != merged_data['Booking_Indirizzo'])]
null_address = merged_data[(merged_data['API_Indirizzo'].isnull()) | (merged_data['Booking_Indirizzo'].isnull())]

Dataset 1 - equal_address

In [ ]:
print("Numero di righe in equal_address:", len(equal_address))
equal_address

Dataset 2 - different_address

In [ ]:
print("Numero di righe in different_address:", len(different_address))
different_address

Dataset 3 - null_address

In [ ]:
print("Numero di righe in null_address:", len(null_address))
null_address

WORK ON THE FIRST DATASET - EQUAL_ADDRESS

In [ ]:
# perform a check on the structure names.
# first look at how many rows have the same name, then structure_name = name.

# first convert the columns 'denominazione_struttura' and 'nome' to lower case, so that the search is "case insensitive"
equal_address['denominazione_struttura'] = equal_address['denominazione_struttura'].str.lower()
equal_address['nome'] = equal_address['nome'].str.lower()

equal_name = equal_address[(equal_address['denominazione_struttura'] == equal_address['nome'])]
equal_name

In [ ]:
# Iterate through the rows of the dataset "equal_address"
# Now we try to check which rows of the dataset have one occurrence (these will be correct)
# even if structure_name and name do not exactly match.
# N.B. here you will also find those with 'denominazione_struttura' = 'nome'!
occorrenze = 0
indici1 = []

for index, row in equal_address.iterrows():
    indirizzo = row['API_Indirizzo']
    denominazione_struttura = row['denominazione_struttura']
    nome = row['nome']

    righe_stesso_indirizzo = equal_address[equal_address['API_Indirizzo'] == indirizzo]

    num_righe_stesso_indirizzo = len(righe_stesso_indirizzo)

    # If there is more than one line with the same address, compare "denominazione_struttura" e "nome"
    if num_righe_stesso_indirizzo == 1:
        # print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}')
        # print('-----------------------------------------')
        occorrenze += 1
        indici1.append(index)

# print(occorrenze)
print(indici1)

In [ ]:
# It happens that there are several facilities that have the same address.
# These are, for example, apartments that are in the same building.
# We look for those that have more than one recurrence (among them we will have to check how similar they are
# and choose the most similar to each other).

conteggio = 0
indici2 = []

for index, row in equal_address.iterrows():
    indirizzo = row['API_Indirizzo']
    denominazione_struttura = row['denominazione_struttura']
    nome = row['nome']

    # Count how many rows share the same address
    righe_stesso_indirizzo = equal_address[equal_address['API_Indirizzo'] == indirizzo]
    num_righe_stesso_indirizzo = len(righe_stesso_indirizzo)

    # If there is more than one line with the same address, compare "denominazione_struttura" and "nome"
    if num_righe_stesso_indirizzo > 1:

        # Split strings into words
        number_d = denominazione_struttura.split()
        number_n = nome.split()

        # Calculate the length of the two strings
        lunghezza_d = len(number_d)
        lunghezza_n = len(number_n)

        # Calculates the similarity between "denominazione_struttura" e "nome"
        similarity = fuzz.token_sort_ratio(denominazione_struttura, nome)

        if lunghezza_d < lunghezza_n:
            if denominazione_struttura in nome or similarity >= 50: # if the similarity is equal or higher than 50%, append the result
                # print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}')
                # print('-----------------------------------------')
                conteggio += 1
                indici2.append(index)
        elif lunghezza_d > lunghezza_n:
            if nome in denominazione_struttura or similarity >= 50:
                # print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}')
                # print('-----------------------------------------')
                conteggio += 1
                indici2.append(index)

# print(conteggio)
print(indici2)

In [ ]:
len(indici1)

In [ ]:
len(indici2)

In [ ]:
indici_equal_address = list(set(indici1 + indici2))
len(indici_equal_address)

In [ ]:
# create a dataset called "equal_address_cleaned"
# containing only the rows in the dataset "equal_address"
# that have the same indexes as those in the list "indici_equal_address"

equal_address_cleaned = equal_address.loc[indici_equal_address]
equal_address_cleaned

WORK ON THE SECOND DATASET - DIFFERENT_ADDRESS

In [ ]:
# I perform an additional check on the addresses of the structures.
# it is indeed possible that they do not match exactly
# because an extra word has been inserted (es. frazione o località)

# Convert columns 'API_Indirizzo' e 'Booking_Indirizzo' lowercase
different_address['API_Indirizzo'] = different_address['API_Indirizzo'].str.lower()
different_address['Booking_Indirizzo'] = different_address['Booking_Indirizzo'].str.lower()
different_address['denominazione_struttura'] = different_address['denominazione_struttura'].str.lower()
different_address['nome'] = different_address['nome'].str.lower()

indici3 = []

for index, row in different_address.iterrows():
    API_Indirizzo = row['API_Indirizzo']
    Booking_Indirizzo = row['Booking_Indirizzo']

    # Check whether values are of type string
    if isinstance(API_Indirizzo, str) and isinstance(Booking_Indirizzo, str):

        # Divide strings into words
        number_a = API_Indirizzo.split()
        number_b = Booking_Indirizzo.split()

        # Calculate the length of the two strings
        lunghezza_a = len(number_a)
        lunghezza_b = len(number_b)

        # Calculate similarity
        similarity = fuzz.token_sort_ratio(API_Indirizzo, Booking_Indirizzo)

        if lunghezza_a < lunghezza_b:
            if API_Indirizzo in Booking_Indirizzo:
                print(f'API_Indirizzo: {API_Indirizzo} \nBooking_Indirizzo: {Booking_Indirizzo}')
                print('-----------------------------------------')
                #conteggio += 1
                indici3.append(index)
        elif lunghezza_a > lunghezza_b:
            if Booking_Indirizzo in API_Indirizzo:
                print(f'API_Indirizzo: {API_Indirizzo} \nBooking_Indirizzo: {Booking_Indirizzo}')
                print('-----------------------------------------')
                #conteggio += 1
                indici3.append(index)
        elif similarity >= 70:
            print(f'API_Indirizzo: {API_Indirizzo} \nBooking_Indirizzo: {Booking_Indirizzo}')
            print('-----------------------------------------')
            indici3.append(index)

print(indici3)

In [ ]:
len(indici3)

In [ ]:
# do a further check on house numbers.
# (keep in mind that those that don't have house numbers will have to be handled some other way)
import re

indici_civici = []
indici_no_civici = []
indici_drop = []

for index in indici3:
    row = different_address.loc[index]
    API_Indirizzo = row['API_Indirizzo']
    Booking_Indirizzo = row['Booking_Indirizzo']
    Denominazione_struttura = row['denominazione_struttura']
    nome = row['nome']

    # Extract the house number from "API_Indirizzo" e "Booking_Indirizzo"
    numero_civico_a = re.findall(r'\d+[\w\s/]*(?=\s|$)', API_Indirizzo)
    numero_civico_b = re.findall(r'\d+[\w\s/]*(?=\s|$)', Booking_Indirizzo)

    # Check if both house numbers are present and are the same.
    if numero_civico_a and numero_civico_b:
        if numero_civico_a == numero_civico_b:
            print(index)
            print(f'API_Indirizzo: {API_Indirizzo} \nBooking_Indirizzo: {Booking_Indirizzo}')
            print(f'Denominazione_struttura: {Denominazione_struttura} \nnome: {nome}')
            print('-----------------------------------------')
            indici_civici.append(index)
        elif numero_civico_a != numero_civico_b:
            print(index)
            print(f'API_Indirizzo: {API_Indirizzo} \nBooking_Indirizzo: {Booking_Indirizzo}')
            print(f'Denominazione_struttura: {Denominazione_struttura} \nnome: {nome}')
            print('-----------------------------------------')
            indici_drop.append(index)
    elif not numero_civico_a or not numero_civico_b:
        indici_no_civici.append(index)

print(indici_civici)
print(indici_no_civici)
print(indici_drop)

In [ ]:
len(indici_civici)

In [ ]:
len(indici_no_civici)

In [ ]:
len(indici_drop)

In [ ]:
# I do a further check on addresses that have house numbers.
indici_civici1 = []

for index, row in different_address.iterrows():
    API_Indirizzo = row['API_Indirizzo']
    Booking_Indirizzo = row['Booking_Indirizzo']
    denominazione_struttura = row['denominazione_struttura']
    nome = row['nome']

    # If there are more than one line with the same address, compare "name_structure" and "name"
    if index in indici_civici:

        # Split strings into words
        number_d = denominazione_struttura.split()
        number_n = nome.split()

        # Calculate the length of the two strings
        lunghezza_d = len(number_d)
        lunghezza_n = len(number_n)

        # Calculates the similarity between "denominazione_struttura" e "nome"
        similarity = fuzz.token_sort_ratio(denominazione_struttura, nome)

        if lunghezza_d < lunghezza_n:
            if denominazione_struttura in nome:
                print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
                print('-----------------------------------------')
                indici_civici1.append(index)
        elif lunghezza_d > lunghezza_n:
            if nome in denominazione_struttura:
                print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
                print('-----------------------------------------')
                indici_civici1.append(index)
        elif similarity >= 50: # If the similarity is greater than or equal to 50%, print and append the results
            print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
            print('-----------------------------------------')
            indici_civici1.append(index)

# print(conteggio)
print(indici_civici1)

In [ ]:
print(len(indici_civici1))

In [ ]:
# I do a further check on addresses that do not have house numbers
indici_no_civici1 = []

for index, row in different_address.iterrows():
    API_Indirizzo = row['API_Indirizzo']
    Booking_Indirizzo = row['Booking_Indirizzo']
    denominazione_struttura = row['denominazione_struttura']
    nome = row['nome']

    if index in indici_no_civici:

        number_d = denominazione_struttura.split()
        number_n = nome.split()

        lunghezza_d = len(number_d)
        lunghezza_n = len(number_n)

        similarity = fuzz.token_sort_ratio(denominazione_struttura, nome)

        if lunghezza_d < lunghezza_n:
            if denominazione_struttura in nome:
                print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
                print('-----------------------------------------')
                indici_no_civici1.append(index)
        elif lunghezza_d > lunghezza_n:
            if nome in denominazione_struttura:
                print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
                print('-----------------------------------------')
                indici_no_civici1.append(index)
        elif similarity >= 50:
            print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
            print('-----------------------------------------')
            indici_no_civici1.append(index)

# print(conteggio)
print(indici_no_civici1)

In [ ]:
print(len(indici_no_civici1))

In [ ]:
# do a further check on addresses that do not have house numbers
indici_drop1 = []

for index, row in different_address.iterrows():
    API_Indirizzo = row['API_Indirizzo']
    Booking_Indirizzo = row['Booking_Indirizzo']
    denominazione_struttura = row['denominazione_struttura']
    nome = row['nome']

    if index in indici_drop:

        number_d = denominazione_struttura.split()
        number_n = nome.split()

        lunghezza_d = len(number_d)
        lunghezza_n = len(number_n)

        similarity = fuzz.token_sort_ratio(denominazione_struttura, nome)

        if lunghezza_d < lunghezza_n:
            if denominazione_struttura in nome:
                print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
                print('-----------------------------------------')
                indici_drop1.append(index)
        elif lunghezza_d > lunghezza_n:
            if nome in denominazione_struttura:
                print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
                print('-----------------------------------------')
                indici_drop1.append(index)
        elif similarity >= 80:
            print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
            print('-----------------------------------------')
            indici_drop1.append(index)

# print(conteggio)
print(indici_drop1)

In [ ]:
print(len(indici_drop1))

In [ ]:
# do a further check on addresses that do not have house numbers
indici_no_drop = []

for index, row in different_address.iterrows():
    API_Indirizzo = row['API_Indirizzo']
    Booking_Indirizzo = row['Booking_Indirizzo']
    denominazione_struttura = row['denominazione_struttura']
    nome = row['nome']

    if index in indici_drop1:

        number_d = denominazione_struttura.split()
        number_n = nome.split()
        number_a = API_Indirizzo.split()
        number_b = Booking_Indirizzo.split()

        lunghezza_d = len(number_d)
        lunghezza_n = len(number_n)
        lunghezza_a = len(number_a)
        lunghezza_b = len(number_b)

        similarity1 = fuzz.token_sort_ratio(API_Indirizzo, Booking_Indirizzo)
        similarity2 = fuzz.token_sort_ratio(denominazione_struttura, nome)

        if similarity1 >= 90 and similarity2 >= 90:
            print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
            print('-----------------------------------------')
            indici_no_drop.append(index)

# print(conteggio)
print(indici_no_drop)

In [ ]:
# drop the 3rd index because it's wrong
indici_no_drop.pop(2)
indici_no_drop

In [ ]:
# sum the lists: indici_civici1, indici_no_civici1, indici_no_drop
indici_different_address = list(set(indici_civici1 + indici_no_civici1 + indici_no_drop))
len(indici_different_address)

In [ ]:
# create a dataset called "different_address_cleaned"
# containing only rows of the dataset "different_address"
# which have the same indexes that are in the list "different_address_cleaned"

different_address_cleaned = different_address.loc[indici_different_address]
different_address_cleaned

WORK ON THE THIRD DATASET - NULL_ADDRESS

In [ ]:
# In this case we cannot rely on the address because
# either in 'API_Indirizzo' or in 'Booking_Indirizzo' or both it is missing.
# Therefore, we can work on the similarity between 'denominazione_struttura' e 'nome' or 'localita' and 'località'

# Convert the columns
null_address['localita'] = null_address['localita'].str.lower()
null_address['località'] = null_address['località'].str.lower()
null_address['denominazione_struttura'] = null_address['denominazione_struttura'].str.lower()
null_address['nome'] = null_address['nome'].str.lower()

indici4 = []

for index, row in null_address.iterrows():
    localita = row['localita']
    località = row['località']
    denominazione_struttura = row['denominazione_struttura']
    nome = row['nome']

    if isinstance(localita, str) and isinstance(località, str):

        number_d = denominazione_struttura.split()
        number_n = nome.split()
        number_a = localita.split()
        number_b = località.split()

        lunghezza_d = len(number_d)
        lunghezza_n = len(number_n)
        lunghezza_a = len(number_a)
        lunghezza_b = len(number_b)

        similarity1 = fuzz.token_sort_ratio(localita, località)
        similarity2 = fuzz.token_sort_ratio(denominazione_struttura, nome)

        if not pd.isna(localita) and not pd.isna(località):
            if similarity1 >= 70 and similarity2 >= 50:
                print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
                print('-----------------------------------------')
                indici4.append(index)
        else:
            if similarity2 >= 50:
                print(f'Denominazione struttura: {denominazione_struttura} \nNome: {nome}\nAPI Indirizzo: {API_Indirizzo}\nBooking Indirizzo: {Booking_Indirizzo}')
                print('-----------------------------------------')
                indici4.append(index)

print(indici4)

In [ ]:
# since I find no matches in this third dataset, then I continue with the merge.
# I merge the previous two results:

merged_df = pd.concat([equal_address_cleaned, different_address_cleaned])

# restore the original indexes, using the reset_index method.
merged_df = merged_df.reset_index(drop=True)

merged_df

In [ ]:
# I update the dataset downloaded from the API and call it "API_complete"

# Create a copy of the api_structure dataframe.
API_completo = api_structure.copy()

# Update the API_complete dataframe using the merged_df dataframe.
API_completo = merged_df.combine_first(API_completo)

In [ ]:
# join the columns of the 'comune'
API_completo['comune'] = API_completo['API_Comune'].combine_first(API_completo['Comune'])
API_completo.drop(['API_Comune', 'Comune'], axis=1, inplace=True)

In [ ]:
# join the columns of the 'indirizzo'
API_completo['Indirizzo'] = API_completo['API_Indirizzo'].combine_first(API_completo['Booking_Indirizzo'])
API_completo.drop(['API_Indirizzo', 'Booking_Indirizzo'], axis=1, inplace=True)

In [ ]:
# join the columns of the 'cap'
API_completo['Cap'] = API_completo['cap'].combine_first(API_completo['CAP'])
API_completo.drop(['CAP', 'cap'], axis=1, inplace=True)

In [ ]:
# join the columns of the 'località'
API_completo['Località'] = API_completo['localita'].combine_first(API_completo['località'])
API_completo.drop(['localita', 'località'], axis=1, inplace=True)

In [ ]:
API_completo

In [ ]:
# Save the DataFrame in 'API_completo.csv'
API_completo.to_csv('API_completo.csv', index=False)